In [ ]:
import pandas as pd
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hybrid features ACPs.csv')

In [ ]:
data.shape

In [ ]:
# Separate the features and target variable
X = data.iloc[:, 0:953].values
y = data.iloc [:, 953].values
print(X.shape)
print(y.shape)

In [ ]:

# Fitness function: Customize it based on your feature selection criteria
def fitness_function(feature_indices):
    # Use feature_indices to filter features and calculate performance metric
    X_selected = X[:, feature_indices]
    # For example, calculate the sum of variances (you can replace this with model accuracy, etc.)
    return np.sum(np.var(X_selected, axis=0))  # minimize variance for simplicity

# Parameters
num_features = X.shape[1]        # Total features in the dataset
num_selected_features = 437      # Number of features to select
num_population = 20              # Number of frogs
num_memplexes = 5                # Number of memplexes
max_iter = 100                   # Maximum iterations
step_size = 1                    # Step size for updating positions

# Initialize frogs randomly with feature indices
population = [np.random.choice(num_features, num_selected_features, replace=False) for _ in range(num_population)]

# Main optimization loop
for iter in range(max_iter):
    # Evaluate fitness
    fitness = np.array([fitness_function(frog) for frog in population])
    sorted_indices = np.argsort(fitness)
    population = [population[i] for i in sorted_indices]

    # Shuffle population and divide into memplexes
    np.random.shuffle(population)
    memplex_size = num_population // num_memplexes
    memplexes = [population[i:i + memplex_size] for i in range(0, num_population, memplex_size)]

    # Local search within each memplex
    for memplex in memplexes:
        best_frog = memplex[0]
        best_fitness = fitness_function(best_frog)

        for frog in memplex:
            # Generate new feature subset (neighboring solution)
            new_indices = frog + np.random.randint(-step_size, step_size + 1, size=num_selected_features)
            new_indices = np.clip(new_indices, 0, num_features - 1)  # Keep indices within range
            new_fitness = fitness_function(new_indices)

            # Update if the new subset is better
            if new_fitness < fitness_function(frog):
                frog[:] = new_indices

    # Combine updated memplexes
    population = [frog for memplex in memplexes for frog in memplex]

# Best solution
best_solution = population[0]
best_fitness = fitness_function(best_solution)
print("Best feature indices:", best_solution)
print("Best fitness:", best_fitness)

In [ ]:
# Save the best selected features in a CSV file
best_features = X[:, best_solution]  # Get the best selected features
df_best_features = pd.DataFrame(best_features)  # Convert to DataFrame for easy CSV saving
df_best_features.to_csv("best_selected_features-SFLA.csv", index=False)  # Save to CSV